In [1]:
import numpy as np
from scipy import stats
import unittest
%run probability_fns.ipynb
%run likelihood_fns.ipynb

...../var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/906992457.py:117: RuntimeWarning: divide by zero encountered in log
  LP += np.log(sigma_prior(sigma, lower_0, upper_0, printing, debug))
/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/906992457.py:373: RuntimeWarning: divide by zero encountered in log
  real_LP = np.sum(np.log(probs))
.....
----------------------------------------------------------------------
Ran 10 tests in 0.011s

OK
...../var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/906992457.py:117: RuntimeWarning: divide by zero encountered in log
  LP += np.log(sigma_prior(sigma, lower_0, upper_0, printing, debug))
/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/906992457.py:373: RuntimeWarning: divide by zero encountered in log
  real_LP = np.sum(np.log(probs))
.....
----------------------------------------------------------------------
Ran 10 tests in 0.010s

OK
......

end is lower than start; can happen and will return 0 which will cause neg infs later: [-inf, 0.0, -1.0, 1.0, inf]



----------------------------------------------------------------------
Ran 6 tests in 0.012s

OK


## Likelihood functions and checks
This notebook is to be used as a submodule that contains wrappers for all the likelihood functions used by the ordinal probit model for survey data, and Metropolis-Hastings sampler. There is also an optional testing suite.

In [2]:
def initialize_suite(TestCase):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestCase)
    return suite 

In [4]:
def mu_accept(mu, mu_star, guessVector, sigma, thetas, mu_0, sigma_0, sigma_prop,
             printing=False, debug=False):
    """Returns the acceptance probability of a proposal for mu.
    
    Inputs
    -------------------
    mu: current latent mean; scalar value in Reals;
    mu_star: proposal for new latent mean; scalar value in Reals;
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    sigma: latent sd; scalar value in (0, inf);
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    
    A1 = joint_log_probability_mu(guessVector, mu_star, sigma, thetas, mu_0, sigma_0, 
                        printing, debug)
    A2 = joint_log_probability_mu(guessVector, mu, sigma, thetas, mu_0, sigma_0, 
                        printing, debug)
    
    A3 = mu_log_jump_probs(mu, mu_star, sigma_prop, printing, debug)
    
    A = A1 - A2 + A3
    return min(1, np.exp(A))

def sigma_accept(sigma, sigma_star, guessVector, mu, thetas, sigma_prop, lower_0, upper_0,
             printing=False, debug=False):
    """Returns the acceptance probability of a proposal for sigma.
    
    Inputs
    -------------------
    sigma: current latent mean; scalar value in (0, inf);
    sigma_star: proposal for new latent mean; scalar value in (0, inf); 
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    mu: latent mean; scalar value in Reals;
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.
    
    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    A1 = joint_log_probability_sigma(guessVector, mu, sigma_star, thetas, lower_0, upper_0,
                        printing, debug)
    A2 = joint_log_probability_sigma(guessVector, mu, sigma, thetas, lower_0, upper_0,
                        printing, debug)
    
    A3 = sigma_log_jump_probs(sigma, sigma_star, sigma_prop, lower_0, upper_0, printing, debug)
    
    A = A1 - A2 + A3
    return min(1, np.exp(A))

def theta_accept(theta, theta_star, thetas, guessMatrix, mus, sigmas, center, shift, sigma_prop, 
                 lower_0, upper_0, printing=False, debug=False):
    """Returns the acceptance probability of a proposal for theta. A bit inefficient. Note, have to monitor
    here for -infs, as sometimes the proposals reverse order sloppily. 
    
    Inputs
    -------------------
    theta: a current threshold; scalar value in (0, inf);
    theta_star: proposal for new latent mean; scalar value in (0, inf); 
    guessVector: guesses / y's. Each should be an integer from 1:k; could be list or vector of ints;
    mu: latent mean; scalar value in Reals;
    thetas: values of latent thresholds defining probit probabilities; scalars in [1.5, k-0.5];
            endpoints are -inf, inf. Length is k+2, where k is the number of possible ordinal choices.
    mu_0: prior mean; scalar value in Reals;
    sigma_0: prior standard devation; scalar value in (0, inf);
    printing: Bool; whether to print messages;
    debug: Bool; whether to run internal error checks.

    Outputs
    -------------------
    AP: acceptance probability (ratio); scalar in (0, inf).
    """
    
    thetasTemp = thetas.copy()
    thetasTemp[center] = theta_star
    
    A1 = joint_log_probability_thetas(guessMatrix, mus, sigmas, thetasTemp, shift, sigma_prop,
                                     printing, debug)
    
    A2 = joint_log_probability_thetas(guessMatrix, mus, sigmas, thetas, shift, sigma_prop,
                                     printing, debug)
    
    A3 = theta_log_jump_probs(theta, theta_star, sigma_prop, lower_0, upper_0, printing, debug)

    A = A1 - A2 + A3
    
    if debug:
        assert -np.inf <= A <= 0.0, "A not in range: A1 {} A2 {} A3{}".format(A1, A2, A3)
        if np.isneginf(A):
            print('A is -inf: A1 {} A2 {} A3{}'.format(A1, A2, A3))
            
    if np.isneginf(A):
        return 0.0 # do not move if that would swap a threshold
    else:
        return min(1, np.exp(A))
    
class LikelihoodFunctionsTestSuite(unittest.TestCase):
    def gauss(self, val, mu=0, s=1):
            return np.exp(- (1/2)*
                          ((val-mu)/s)**2) / np.sqrt(2*np.pi*(s**2))
    
    def gauss_trunc(self, x, mu, sigma, lower, upper):
        zeta = (x-mu)/sigma
        alpha = (lower-mu)/sigma
        beta = (upper-mu)/sigma
        Z = stats.norm.cdf(beta) - stats.norm.cdf(alpha)
        return (self.gauss(zeta)) / (sigma*Z)
    
    def test_mu_accept(self):
        #             mu. mu_star  y.    sigma         thetas             mu_0 s_0  s_j
        testValues = [0.0, 1.0, [2, 1], 2.0, [-np.inf, 1.5, 2.5, np.inf], 1.0, 1.0, 2.0]
        mu, mu_star, guessVector, sigma, thetas, mu_0, s_0, s_j = testValues.copy()
        
        returned_A = mu_accept(*testValues, printing=True, debug=True)
        
        real_A1 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu_star, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu_star, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu_star, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu_star, sigma)),
                                self.gauss(mu_star, mu_0, s_0)]))
        real_A2 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma)),
                                self.gauss(mu, mu_0, s_0)]))
        
        real_A3 = 0
        comb = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A, comb)
        
        
    def test_sigma_accept(self):
        #             s.  s_star  y.    mu         thetas                s_j l1 l2
        testValues = [1.0, 2.0, [2, 1], 2.0, [-np.inf, 1.5, 2.5, np.inf], 2.0, 0, 3]
        sigma, sigma_star, guessVector, mu, thetas, s_j, l1, l2 = testValues.copy()
        returned_A = sigma_accept(*testValues, printing=True, debug=True)
        real_A1 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma_star) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma_star)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma_star) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma_star)),
                                1/(l2-l1)]))
        real_A2 = np.sum(np.log([max(0, stats.norm.cdf(thetas[guessVector[0]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[0]], mu, sigma)), 
                                max(0, stats.norm.cdf(thetas[guessVector[1]+1], mu, sigma) - stats.norm.cdf(thetas[guessVector[1]], mu, sigma)),
                                1/(l2-l1)]))
        real_A3 = np.log(self.gauss_trunc(sigma, sigma_star, s_j, l1, l2)) - np.log(self.gauss_trunc(sigma_star, sigma, s_j, l1, l2)) # because symmetric ---> Metropolis
        comb = np.min([1, np.exp(real_A1 - real_A2 + real_A3)])
        self.assertAlmostEqual(returned_A, comb)
        
    def test_theta_accept(self):
        testValues = [2.5, 2.6, [-np.inf, 1.5, 2.5, 3.5, np.inf], [[2, 1], [2, 3]], [1.0, 2.0], [1.0, 1.0], 2, 0.5, 2.0, 0, 3]
        theta, theta_star, thetas, guessMatrix, mus, sigmas, center, shift, sigma_prop, lower_0, upper_0 = testValues.copy()
        returned_A = theta_accept(*testValues, printing=True, debug=True)
    
    
runner = unittest.TextTestRunner(failfast=True)
runner.run(initialize_suite(LikelihoodFunctionsTestSuite))  

..F
FAIL: test_theta_accept (__main__.LikelihoodFunctionsTestSuite)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/1685385032.py", line 159, in test_theta_accept
    returned_A = theta_accept(*testValues, printing=True, debug=True)
  File "/var/folders/z3/65sxpv5n3csdhfzm0ysx30940000gp/T/ipykernel_44869/1685385032.py", line 101, in theta_accept
    assert -np.inf <= A <= 0.0, "A not in range: A1 {} A2 {} A3{}".format(A1, A2, A3)
AssertionError: A not in range: A1 -9.024957400482823 A2 -9.004984854591386 A30.02172746830337635

----------------------------------------------------------------------
Ran 3 tests in 0.009s

FAILED (failures=1)


<unittest.runner.TextTestResult run=3 errors=0 failures=1>